# Interior Mutability

In [2]:
use std::collections::HashMap;

struct DnsService {
    lookup_table: HashMap<String, String>
}

impl DnsService {
    fn lookup(&self, hostname: &str) -> Option<&str> {
        self.lookup_table.get(hostname).map(|s| s.as_str())
    }
}

In [3]:
let dns_service = DnsService {
    lookup_table: [("www.example.com".to_string(), "143.33.44.4".to_string()),
            ("www.rust-lang.org".to_string(), "1.22.33.1".to_string()),
            ("www.mozilla.org".to_string(), "2.43.44.3".to_string())
        ].iter().cloned().collect()
};

println!("{:?}", dns_service.lookup("www.rust-lang.org"));

Some("1.22.33.1")


In [4]:
use std::collections::HashMap;

struct DnsService {
    lookup_table: HashMap<String, String>,
    counter: i32
}

impl DnsService {
    fn lookup(&self, hostname: &str) -> Option<&str> {
        self.counter += 1; // Error: cannot assign to `self.counter` because it is borrowed
        self.lookup_table.get(hostname).map(|s| s.as_str())
    }
}

Error: cannot assign to `self.counter`, which is behind a `&` reference

## `Cell` - for `Copy` types

In [5]:
use std::collections::HashMap;
use std::cell::Cell;

struct DnsService {
    lookup_table: HashMap<String, String>,
    counter: Cell<i32>
}

impl DnsService {
    fn lookup(&self, hostname: &str) -> Option<&str> {
        let value = self.counter.get();
        self.counter.set(value + 1);
        self.lookup_table.get(hostname).map(|s| s.as_str())
    }
}

In [6]:
let dns_service = DnsService {
    lookup_table: [("www.example.com".to_string(), "143.33.44.4".to_string()),
            ("www.rust-lang.org".to_string(), "1.22.33.1".to_string()),
            ("www.mozilla.org".to_string(), "2.43.44.3".to_string())
        ].iter().cloned().collect(), 
        counter: Cell::new(0)
};

println!("{:?}", dns_service.lookup("www.rust-lang.org"));
println!("{:?}", dns_service.lookup("www.example.com"));
println!("{:?}", dns_service.lookup("www.infotraining.pl"));

println!("Dns used {} times", dns_service.counter.get());

The type of the variable dns_service was redefined, so was lost.


Some("1.22.33.1")
Some("143.33.44.4")
None
Dns used 3 times


## RefCell - for non-`Copy` types

In [7]:
use std::collections::HashMap;
use std::cell::RefCell;

type LookupLog = Vec<String>;

struct DnsService {
    lookup_table: HashMap<String, String>,
    lookup_log: RefCell<LookupLog>,
}

impl DnsService {
    fn lookup(&self, hostname: &str) -> Option<&str> {
        self.lookup_log.borrow_mut().push(hostname.to_string());
        self.lookup_table.get(hostname).map(|s| s.as_str())
    }
}

In [8]:
let dns_service = DnsService {
    lookup_table: [("www.example.com".to_string(), "143.33.44.4".to_string()),
            ("www.rust-lang.org".to_string(), "1.22.33.1".to_string()),
            ("www.mozilla.org".to_string(), "2.43.44.3".to_string())
        ].iter().cloned().collect(), 
    lookup_log: RefCell::new(Vec::new())
};

println!("{:?}", dns_service.lookup("www.rust-lang.org"));
println!("{:?}", dns_service.lookup("www.example.com"));
println!("{:?}", dns_service.lookup("www.infotraining.pl"));

println!("Dns looked up for {:?}", dns_service.lookup_log.borrow());

Some("1.22.33.1")
Some("143.33.44.4")
None
Dns looked up for ["www.rust-lang.org", "www.example.com", "www.infotraining.pl"]


## RwLock - for thread-safe shared access

In [9]:
use std::collections::HashMap;
use std::cell::RefCell;

type LookupLog = Vec<String>;

struct DnsService {
    lookup_table: HashMap<String, String>,
    lookup_log: RefCell<LookupLog>,
}

impl DnsService {
    fn from_slice(data: &[(&str, &str)]) -> DnsService {
        DnsService {
            lookup_table: data.iter().cloned().map(|(k, v)| (k.to_string(), v.to_string())).collect(),
            lookup_log: RefCell::new(Vec::new())
        }
    }

    fn lookup(&self, hostname: &str) -> Option<&str> {
        self.lookup_log.borrow_mut().push(hostname.to_string());
        self.lookup_table.get(hostname).map(|s| s.as_str())
    }
}

In [11]:
fn main() {
    let dns_service = DnsService::from_slice(&[("www.example.com", "122.33.22.33"), ("www.rust-lang.org", "1.2.3.4"), ("www.mozilla.org", "9.4.3.4")]);

    println!("{:?}", dns_service.lookup("www.rust-lang.org"));

    let thd_1 = std::thread::spawn(|| {
        println!("{:?}", dns_service.lookup("www.example.com"));
    });

    thd_1.join().unwrap();
}

main();

Error: `RefCell<Vec<String>>` cannot be shared between threads safely

## Arc + RwLock - for shared access across threads

In [12]:
use std::collections::HashMap;
use std::sync::RwLock;
use std::borrow::BorrowMut;

type LookupLog = Vec<String>;

struct DnsService {
    lookup_table: HashMap<String, String>,
    lookup_log: RwLock<LookupLog>,
}

impl DnsService {
    fn from_slice(data: &[(&str, &str)]) -> DnsService {
        DnsService {
            lookup_table: data.iter().cloned().map(|(k, v)| (k.to_string(), v.to_string())).collect(),
            lookup_log: RwLock::new(Vec::new())
        }
    }

    fn lookup(&self, hostname: &str) -> Option<&str> {
        let mut log = self.lookup_log.write().unwrap();
        log.push(hostname.to_string());
        self.lookup_table.get(hostname).map(|s| s.as_str())
    }
}

In [13]:
use std::sync::Arc;

fn main() {
    let dns_service = Arc::new(
        DnsService::from_slice(&[("www.example.com", "122.33.22.33"), ("www.rust-lang.org", "1.2.3.4"), ("www.mozilla.org", "9.4.3.4")])
    );

    println!("{:?}", dns_service.lookup("www.rust-lang.org"));

    let mut dns = dns_service.clone();
    let thd_1 = std::thread::spawn(move || {
        println!("{:?}", dns.lookup("www.example.com"));
    });

    let mut dns = dns_service.clone();
    let thd_2 = std::thread::spawn(move || {
        println!("{:?}", dns.lookup("www.mozilla.org"));
    });

    thd_1.join().unwrap();
    thd_2.join().unwrap();

    println!("Dns looked up for {:?}", dns_service.lookup_log.read().unwrap());
}

main();

Some("1.2.3.4")
Some("122.33.22.33")
Some("9.4.3.4")
Dns looked up for ["www.rust-lang.org", "www.example.com", "www.mozilla.org"]
